<a href="https://colab.research.google.com/github/jkordonis/TropicalML/blob/main/MNISTandTropDivision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import math
from scipy.linalg import block_diag
import os
import io
import cvxpy as cp




```
# This is formatted as code
```

# MNIST Train from Keras

In [6]:
num_classes = 10 # Initially 10 classes
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
Ner=100
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Flatten(),
        layers.Dense(Ner, activation="relu"),     
        layers.Dense(10, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [8]:
batch_size = 128
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 2s 4ms/step - loss: 0.4094 - accuracy: 0.8888 - val_loss: 0.1914 - val_accuracy: 0.9505
Epoch 2/10
422/422 [==============================] - 2s 4ms/step - loss: 0.1966 - accuracy: 0.9449 - val_loss: 0.1392 - val_accuracy: 0.9618
Epoch 3/10
422/422 [==============================] - 2s 5ms/step - loss: 0.1433 - accuracy: 0.9591 - val_loss: 0.1143 - val_accuracy: 0.9680
Epoch 4/10
422/422 [==============================] - 2s 4ms/step - loss: 0.1137 - accuracy: 0.9680 - val_loss: 0.0998 - val_accuracy: 0.9702
Epoch 5/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0929 - accuracy: 0.9736 - val_loss: 0.0926 - val_accuracy: 0.9722
Epoch 6/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0775 - accuracy: 0.9782 - val_loss: 0.0887 - val_accuracy: 0.9738
Epoch 7/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0663 - accuracy: 0.9813 - val_loss: 0.0860 - val_accuracy: 0.9747
Epoch 

In [9]:
W1=model.layers[1].weights[0].numpy()
b1=model.layers[1].weights[1].numpy()
W2=model.layers[2].weights[0].numpy()
b2=model.layers[2].weights[1].numpy()



```
# This is formatted as code
```

# Reshaping Data for 3-5 distinction and Tropical Polynomials

In [10]:
x_test_vec =np.zeros([10000, 784])
for i in range(10000):
  b = np.reshape(x_test[i],(784))
  x_test_vec[i]=b

err=0
y_pred=np.zeros((10000,10))
for i in range(10000):
  z =  np.matmul(W1.T,x_test_vec[i]) +b1
  z=np.maximum(z, np.zeros(Ner))
  y=np.matmul(W2.T,z)+b2
  y_pred[i]=y
  err_i=0
  if np.inner(y_test[i],[0,0,0,1,0,1,0,0,0,0])==1:
    if np.argmax(y)!=np.argmax(y_test[i]):
      err_i=1
  err=err+err_i
err

73

In [11]:
x_test_3_5=[]
y_test_3_5=[]


for i in range(10000):
  if np.inner(y_test[i],[0,0,0,1,0,1,0,0,0,0])==1:
    x_test_3_5.append(x_test_vec[i])
    y_test_3_5.append(-1)
    if np.inner(y_test[i],[0,0,0,1,0,0,0,0,0,0])==1:
      y_test_3_5[-1]=1
    if np.inner(y_test[i],[0,0,0,0,0,1,0,0,0,0])==1:
      y_test_3_5[-1]=0

x_test_3_5=np.array(x_test_3_5)
y_test_3_5=np.array(y_test_3_5)


x_train_3_5=[]
y_train_3_5=[]
x_train_vec =np.zeros([60000, 784])
for i in range(60000):
  b = np.reshape(x_train[i],(784))
  x_train_vec[i]=b



for i in range(60000):
  if np.inner(y_train[i],[0,0,0,1,0,1,0,0,0,0])==1:
    x_train_3_5.append(x_train_vec[i])
    y_train_3_5.append(-1)
    if np.inner(y_train[i],[0,0,0,1,0,0,0,0,0,0])==1:
      y_train_3_5[-1]=1
    if np.inner(y_train[i],[0,0,0,0,0,1,0,0,0,0])==1:
      y_train_3_5[-1]=0

x_train_3_5=np.array(x_train_3_5)
y_train_3_5=np.array(y_train_3_5)


In [12]:
# 3-5 IniOriginal  Error

err=0
for i in range(x_test_3_5.shape[0]):
  z =  np.matmul(W1.T,x_test_3_5[i]) +b1
  z=np.maximum(z, np.zeros(Ner))
  y=np.matmul(W2.T,z)+b2
  err_i=0
  if y[3]>y[5] and y_test_3_5[i]==0:#if np.argmax(y)!=5 and y_test_3_5[i]==0:#
    err_i=1
  if y[3]<y[5] and y_test_3_5[i]==1:#if np.argmax(y)!=3 and y_test_3_5[i]==1:#
    err_i=1
  err=err+err_i
err/x_test_3_5.shape[0]

0.012092534174553101

In [13]:
# Dimension Reduction
W2_=W2[:,3]-W2[:,5]
b2_=b2[3]-b2[5]

err=0
for i in range(x_test_3_5.shape[0]):
  z =  np.matmul(W1.T,x_test_3_5[i]) +b1
  z=np.maximum(z, np.zeros(Ner))
  y=np.matmul(W2_.T,z)+b2_
  err_i=0
  if y>0 and y_test_3_5[i]==0:#if y[3]>y[5] and y_test_3_5[i]==0:
    err_i=1
  if y<0 and y_test_3_5[i]==1:#if y[3]<y[5] and y_test_3_5[i]==1:
    err_i=1
  err=err+err_i
err/x_test_3_5.shape[0]

0.012092534174553101

In [14]:
W2_pl=np.maximum(W2_,np.zeros(W2_.shape))
W2_min=np.maximum(-W2_,np.zeros(W2_.shape))
ap_1 = np.zeros((W1.shape[1],W1.shape[0]))
bp_1 = np.zeros(W1.shape[1])
ap_2 = np.zeros((W1.shape[1],W1.shape[0]))
bp_2 = np.zeros(W1.shape[1])


for i in range(W1.shape[1]):
  ap_1[i]=W2_pl[i]*W1.T[i]
  bp_1[i]=W2_pl[i]*b1[i]
  ap_2[i]=W2_min[i]*W1.T[i]
  bp_2[i]=W2_min[i]*b1[i]

# QR Dimensionality Reduction 

```
# This is formatted as code
```



In [15]:
Q1,R1=np.linalg.qr(ap_1.T,mode='reduced')
Q2,R2=np.linalg.qr(ap_2.T,mode='reduced')
 
a1_vec=ap_1.T
a1_red_vec = np.matmul(Q1.T,a1_vec)
a2_vec=ap_2.T
a2_red_vec = np.matmul(Q2.T,a2_vec)

X_sample1 = np.matmul(x_train_3_5,Q1) 
X_sample2 = np.matmul(x_train_3_5,Q2) 


X_sample1_test = np.matmul(x_test_3_5,Q1) 
X_sample2_test = np.matmul(x_test_3_5,Q2) 

m_q=10



In [16]:
err=0

for i in range(X_sample1_test.shape[0]):
  p1_val = np.sum( np.maximum(np.matmul(a1_red_vec.T,X_sample1_test[i]) +bp_1,np.zeros(ap_1.shape[0])))
  p2_val =  np.sum(np.maximum(np.matmul(a2_red_vec.T,X_sample2_test[i]) +bp_2,np.zeros(ap_1.shape[0])))
  p_val =p1_val-p2_val+b2_
  y=p_val
  err_i=0
  if y>0 and y_test_3_5[i]==0:#if y[3]>y[5] and y_test_3_5[i]==0:
    err_i=1
  if y<0 and y_test_3_5[i]==1:#if y[3]<y[5] and y_test_3_5[i]==1:
    err_i=1
  err=err+err_i
err/X_sample1_test.shape[0]



0.012092534174553101

In [17]:
err=0
for i in range(X_sample1.shape[0]):
  p1_val = np.sum( np.maximum(np.matmul(a1_red_vec.T,X_sample1[i]) +bp_1,np.zeros(ap_1.shape[0])))
  p2_val =  np.sum(np.maximum(np.matmul(a2_red_vec.T,X_sample2[i]) +bp_2,np.zeros(ap_1.shape[0])))
  p_val =p1_val-p2_val+b2_
  y=p_val
  err_i=0
  if y>0 and y_train_3_5[i]==0:#if y[3]>y[5] and y_test_3_5[i]==0:
    err_i=1
  if y<0 and y_train_3_5[i]==1:#if y[3]<y[5] and y_test_3_5[i]==1:
    err_i=1
  err=err+err_i
err/x_train_3_5.shape[0]#Training error(<Test error)

0.004241689750692521

#  Tropical Division 1

In [35]:
a=a1_red_vec.T
b=bp_1
 
d=a.shape[1]
m_p=a.shape[0]



a_til, b_til=np.array([np.zeros(d)]), np.array([0])
a_hat, b_hat=np.array(np.zeros((m_q,d))), np.array(np.zeros(m_q))
m_d=a_til.shape[0]

p_pol=(a,b)
d_pol=(a_til,b_til)
q_pol=(a_hat,b_hat)

# Samples X_sample
X_sample=X_sample1[1:200]
N_sample=X_sample1.shape[0]


 


In [36]:
def tropical_pol_function(x,pol):
  a_=pol[0]
  b_=pol[1]
  s= -math.inf
  for i in range(np.shape(a_)[0]):
    if s<np.inner(a_[i],x)+b_[i]:
      s=np.inner(a_[i],x)+b_[i]
  return(s)
def tropical_sum_pol_function(x,pol):
  a_=pol[0]
  b_=pol[1]
  s= 0
  for i in range(np.shape(a_)[0]):
      s=s+max(np.inner(a_[i],x)+b_[i],0)
  return(s)

In [37]:
f_x_i=np.zeros(np.shape(X_sample)[0])
for i in range(np.shape(X_sample)[0]):
  f_x_i[i]=tropical_sum_pol_function(X_sample[i],p_pol)-tropical_pol_function(X_sample[i],d_pol)

In [38]:
# Phase 1
def Phase_1_function(X_sample,q_pol):
  a_hat,b_hat=q_pol
  # Initialize sets I_i
  I_i = np.zeros(np.shape(X_sample)[0])
  for i in range(np.shape(X_sample)[0]):
    q_pol_values = np.matmul( a_hat,X_sample[i])+b_hat      
    I_i[i]=np.argmax(q_pol_values )
  return I_i

In [39]:
def Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol):
  a_hat, b_hat=q_pol
  s=np.zeros((np.shape(a_hat)[0],np.shape(X_sample)[1]))
  N=np.zeros((np.shape(a_hat)[0]))
  for i in range(np.shape(a_hat)[0]):
    for j in range(np.shape(X_sample)[0]):
      if I_i[j]==i:
        s[i]=s[i]+X_sample[j]
        N[i]=N[i]+1
  return (s, N)

In [40]:
# Formating Matrices
I_d=np.identity(d)
A_diagonal=a.T
Iden_vert=I_d
a_til_vect=a_til[0]
for i in range(m_d-1):
  Iden_vert=np.concatenate((Iden_vert, I_d), axis=0)
  a_til_vect=np.concatenate((a_til_vect, a_til[i+1]), axis=0)
for i in range(m_d-1):
  A_diagonal=block_diag(A_diagonal,a.T)


x_a = cp.Variable(d)
x_b = cp.Variable(1)
l = cp.Variable(m_d*m_p)

l_dim=m_p*m_d
Iterations=15
progress_mat=np.zeros(Iterations+1)
progress_mat[0]=np.sum(f_x_i)-np.sum([tropical_pol_function(X_sample[i],q_pol) for i in range(X_sample.shape[0])])
# Main Iteration
for cnt in range(Iterations):
  print(cnt+1)
  q_pol = (a_hat,b_hat)
  I_i=Phase_1_function(X_sample,q_pol)
  s,N=Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol)
  for i in range(a_hat.shape[0]):
    s_i=s[i]
    N_i=N[i]
    prob = cp.Problem(cp.Maximize(s_i.T@x_a+N_i*x_b),
                    [X_sample@x_a+x_b<=f_x_i,
                     Iden_vert@x_a+a_til_vect==A_diagonal@l,
                     np.zeros(l_dim)<=l,
                     np.ones(l_dim)>=l])
    prob.solve(warm_start=True)
    a_hat[i]=x_a.value
    b_hat[i]=x_b.value
    progress_mat[cnt+1]=np.sum(f_x_i)-np.sum([tropical_pol_function(X_sample[i],q_pol) for i in range(X_sample.shape[0])])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [41]:
progress_mat1=progress_mat
a_hat_1=a_hat
b_hat_1=b_hat

# Tropical Division 2

In [42]:
a=a2_red_vec.T
b=bp_2
 
d=a.shape[1]
m_p=a.shape[0]



a_til, b_til=np.array([np.zeros(d)]), np.array([0])
a_hat, b_hat=np.array(np.zeros((m_q,d))), np.array(np.zeros(m_q))

p_pol=(a,b)
d_pol=(a_til,b_til)
q_pol=(a_hat,b_hat)

# Samples X_sample
X_sample=X_sample2[1:200]
N_sample=X_sample2.shape[0]

In [43]:
def tropical_pol_function(x,pol):
  a_=pol[0]
  b_=pol[1]
  s= -math.inf
  for i in range(np.shape(a_)[0]):
    if s<np.inner(a_[i],x)+b_[i]:
      s=np.inner(a_[i],x)+b_[i]
  return(s)
def tropical_sum_pol_function(x,pol):
  a_=pol[0]
  b_=pol[1]
  s= 0
  for i in range(np.shape(a_)[0]):
      s=s+max(np.inner(a_[i],x)+b_[i],0)
  return(s)

In [44]:
f_x_i=np.zeros(np.shape(X_sample)[0])
for i in range(np.shape(X_sample)[0]):
  f_x_i[i]=tropical_sum_pol_function(X_sample[i],p_pol)-tropical_pol_function(X_sample[i],d_pol)

In [45]:
# Phase 1
def Phase_1_function(X_sample,q_pol):
  a_hat,b_hat=q_pol
  # Initialize sets I_i
  I_i = np.zeros(np.shape(X_sample)[0])
  for i in range(np.shape(X_sample)[0]):
    q_pol_values = np.matmul( a_hat,X_sample[i])+b_hat      
    I_i[i]=np.argmax(q_pol_values )
  return I_i

In [46]:
def Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol):
  a_hat, b_hat=q_pol
  s=np.zeros((np.shape(a_hat)[0],np.shape(X_sample)[1]))
  N=np.zeros((np.shape(a_hat)[0]))
  for i in range(np.shape(a_hat)[0]):
    for j in range(np.shape(X_sample)[0]):
      if I_i[j]==i:
        s[i]=s[i]+X_sample[j]
        N[i]=N[i]+1
  return (s, N)

In [47]:
# Formating Matrices
I_d=np.identity(d)
A_diagonal=a.T
Iden_vert=I_d
a_til_vect=a_til[0]
for i in range(m_d-1):
  Iden_vert=np.concatenate((Iden_vert, I_d), axis=0)
  a_til_vect=np.concatenate((a_til_vect, a_til[i+1]), axis=0)
for i in range(m_d-1):
  A_diagonal=block_diag(A_diagonal,a.T)


x_a = cp.Variable(d)
x_b = cp.Variable(1)
l = cp.Variable(m_d*m_p)

l_dim=m_p*m_d
Iterations=15
progress_mat=np.zeros(Iterations+1)
progress_mat[0]=np.sum(f_x_i)-np.sum([tropical_pol_function(X_sample[i],q_pol) for i in range(X_sample.shape[0])])
# Main Iteration
for cnt in range(Iterations):
  print(cnt+1)
  q_pol = (a_hat,b_hat)
  I_i=Phase_1_function(X_sample,q_pol)
  s,N=Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol)
  for i in range(a_hat.shape[0]):
    s_i=s[i]
    N_i=N[i]
    prob = cp.Problem(cp.Maximize(s_i.T@x_a+N_i*x_b),
                    [X_sample@x_a+x_b<=f_x_i,
                     Iden_vert@x_a+a_til_vect==A_diagonal@l,
                     np.zeros(l_dim)<=l,
                     np.ones(l_dim)>=l])
    prob.solve(warm_start=True)
    a_hat[i]=x_a.value
    b_hat[i]=x_b.value
    progress_mat[cnt+1]=np.sum(f_x_i)-np.sum([tropical_pol_function(X_sample[i],q_pol) for i in range(X_sample.shape[0])])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [48]:
progress_mat2=progress_mat
a_hat_2=a_hat
b_hat_2=b_hat

# Check1

In [49]:
err=0

for i in range(X_sample.shape[0]):
  p1_val = np.max( np.maximum(np.matmul(a_hat_1,X_sample1[i]) +b_hat_1,np.zeros(a_hat.shape[0])))
  p2_val =  np.sum(np.maximum(np.matmul(a2_red_vec.T,X_sample2[i]) +bp_2,np.zeros(ap_1.shape[0])))
  p_val =p1_val-p2_val+b2_
  y=p_val
  err_i=0
  if y>0 and y_train_3_5[i]==0:#if y[3]>y[5] and y_test_3_5[i]==0:
    err_i=1
  if y<0 and y_train_3_5[i]==1:#if y[3]<y[5] and y_test_3_5[i]==1:
    err_i=1
  err=err+err_i
err/X_sample.shape[0]


0.010050251256281407

In [50]:
X_sample1 = np.matmul(x_train_3_5,Q1) 
X_sample2 = np.matmul(x_train_3_5,Q2) 


X_sample1_test = np.matmul(x_test_3_5,Q1) 
X_sample2_test = np.matmul(x_test_3_5,Q2) 

In [51]:
err=0

for i in range(X_sample1_test.shape[0]):
  p1_val = np.max( np.maximum(np.matmul(a_hat_1,X_sample1_test[i]) +b_hat_1,np.zeros(a_hat.shape[0])))
  p2_val = np.max( np.maximum(np.matmul(a_hat_2,X_sample2_test[i]) +b_hat_2,np.zeros(a_hat.shape[0])))
  p_val =p1_val-p2_val+b2_
  y=p_val
  err_i=0
  if y>0 and y_test_3_5[i]==0:#if y[3]>y[5] and y_test_3_5[i]==0:
    err_i=1
  if y<0 and y_test_3_5[i]==1:#if y[3]<y[5] and y_test_3_5[i]==1:
    err_i=1
  err=err+err_i
err/X_sample1_test.shape[0]

0.02050473186119874

In [52]:
err=0

for i in range(200):
  p1_val = np.max( np.maximum(np.matmul(a_hat_1,X_sample1[i]) +b_hat_1,np.zeros(a_hat.shape[0])))
  p2_val = np.max( np.maximum(np.matmul(a_hat_2,X_sample2[i]) +b_hat_2,np.zeros(a_hat.shape[0])))
  p_val =p1_val-p2_val+b2_
  y=p_val
  err_i=0
  if y>0 and y_train_3_5[i]==0:#if y[3]>y[5] and y_test_3_5[i]==0:
    err_i=1
  if y<0 and y_train_3_5[i]==1:#if y[3]<y[5] and y_test_3_5[i]==1:
    err_i=1
  err=err+err_i
err/200

0.015

In [53]:
progress_mat1



array([3090.89797043,  563.84602294,  377.10820379,  342.9675346 ,
        269.63432696,  184.36716044,  155.58979372,  139.86988304,
        134.30141569,  132.74454235,  126.02932969,  113.72876774,
        110.75357911,  110.24943957,  110.09974437,  110.09974406])